In [2]:
import numpy as np 
import pandas as pd

In [11]:
import requests
from bs4 import BeautifulSoup
get_data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
data_xml = BeautifulSoup(get_data, 'xml')
data_table=data_xml.find('table')

_______________________________________

# Part 1 #

_______________________________________

In [12]:
# Creat a dataframe
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)
for tr in data_table.find_all('tr'):
    row_data=[]
    for td in tr.find_all('td'):
        row_data.append(td.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [13]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


**Ignore cells with a borough that is Not assigned**

In [18]:
df=df[df['Borough']!='Not assigned']
df=df[df['Neighborhood']!='Not assigned']
df_new=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
df_new=df_new.reset_index(drop=False)
df_new.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)
df_prune = pd.merge(df, df_new, on='Postalcode')
df_prune.drop(['Neighborhood'],axis=1,inplace=True)
df_prune.drop_duplicates(inplace=True)
df_prune.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)
df_prune.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M7A,Downtown Toronto,Queen's Park


**Shape of dataframe**

In [21]:
df_prune.shape

(103, 3)

_________________________

# Part 2 #

_________________________

In [29]:
df_geo=pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [30]:
df_geo.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_prune = pd.merge(df_geo, df_prune, on='Postalcode')
geo_data=geo_prune[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]
geo_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


_______

# Part 3 #


_________

**Get data in Toronto**

In [31]:
df_toronto=geo_data[geo_data['Borough'].str.contains("Toronto")]
df_toronto.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


**Get queries from FoursquareAPI by credential information**

In [32]:
CLIENT_ID = 'W0NEEQAZIGOMPCSXQILJZ3AL2BJJ4TZBXDCIMW5H2LKKUWO5' 
CLIENT_SECRET = 'K2XR2A4PZVPJEPLCKGJLMNEADID3234NR4MRBME0QMR2OEGV' 
VERSION = '20200311'

**Create function to get postal codes and locations of neighborhoods in Toronto**

In [33]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [34]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

In [37]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [38]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton, Exhibition Place, Parkdale Village",24,24,24,24,24,24
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",17,17,17,17,17,17
"Cabbagetown, St. James Town",46,46,46,46,46,46
Central Bay Street,81,81,81,81,81,81
"Chinatown, Grange Park, Kensington Market",86,86,86,86,86,86
Christie,18,18,18,18,18,18


**One hot code representation**

In [39]:
df_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
df_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
df_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'] )
df_onehot.shape

(1721, 232)

**Group venues in Toronto by Neighborhood**

In [41]:
df_cluster = df_onehot.groupby('Neighborhood').mean().reset_index()
df_cluster.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,0.0,...,0.0,0.0,0.020000,0.0,0.0,0.01,0.0,0.0,0.01,0.000000
1,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.017857,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.041667
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.058824
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.00,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.000000


**Finding top 10 common venues in Toronto**

In [42]:
top = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(top):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
def most_appeared_venues(row, top):
    categories = row.iloc[1:]
    categories_new = categories.sort_values(ascending=False)
    
    return categories_new.index.values[0:top]
neighborhoods = pd.DataFrame(columns=columns)
neighborhoods['Neighborhood'] = df_cluster['Neighborhood']

for i in np.arange(df_cluster.shape[0]):
    neighborhoods.iloc[i, 1:] = most_appeared_venues(df_cluster.iloc[ind, :], top)

neighborhoods.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Gay Bar,Burger Joint,Café,Fast Food Restaurant,Hotel,Mediterranean Restaurant
1,Berczy Park,Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Gay Bar,Burger Joint,Café,Fast Food Restaurant,Hotel,Mediterranean Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Gay Bar,Burger Joint,Café,Fast Food Restaurant,Hotel,Mediterranean Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Gay Bar,Burger Joint,Café,Fast Food Restaurant,Hotel,Mediterranean Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Gay Bar,Burger Joint,Café,Fast Food Restaurant,Hotel,Mediterranean Restaurant


**Define labels for clusters**

In [51]:
from sklearn.cluster import KMeans
k = 5
df_clustering = df_cluster.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=k, random_state=None).fit(df_clustering)
kmeans.labels_[0:10]
toronto_join = df_toronto
toronto_join = toronto_join.join(neighborhoods.set_index('Neighborhood'), on='Neighborhood')

toronto_join.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Gay Bar,Burger Joint,Café,Fast Food Restaurant,Hotel,Mediterranean Restaurant
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Gay Bar,Burger Joint,Café,Fast Food Restaurant,Hotel,Mediterranean Restaurant
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Gay Bar,Burger Joint,Café,Fast Food Restaurant,Hotel,Mediterranean Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Gay Bar,Burger Joint,Café,Fast Food Restaurant,Hotel,Mediterranean Restaurant
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Gay Bar,Burger Joint,Café,Fast Food Restaurant,Hotel,Mediterranean Restaurant


**Print out geographical location of Toronto**

In [54]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode('Toronto, CA')
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


**Construct the map of Toronto**

In [58]:
!conda install -c conda-forge folium=0.5.0 --yes 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

In [62]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_join['Latitude'], toronto_join['Longitude'], toronto_join['Neighborhood'], toronto_join['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters